In [1]:
try:
    from google.cloud import aiplatform

except ImportError:
    !pip3 install -U google-cloud-aiplatform --user

    print("Please restart the kernel and re-run the notebook.")

In [2]:
import os
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

In [3]:
%%bash
PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-00-f6858b4e5c0c


In [4]:
# Change these to try this notebook out
PROJECT = "qwiklabs-gcp-00-f6858b4e5c0c"  # Replace with your PROJECT
BUCKET = PROJECT  # defaults to PROJECT
REGION = "us-east1"  # Replace with your REGION

In [5]:
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

In [6]:
%%bash
gcloud config set project $PROJECT
gcloud config set ai/region $REGION

Updated property [core/project].
Updated property [ai/region].


In [7]:
%%bash
gsutil ls gs://${BUCKET}/energy_forecasting/dnn_normalized_fc*

gs://qwiklabs-gcp-00-f6858b4e5c0c/energy_forecasting/dnn_normalized_fc/saved_model.pb
gs://qwiklabs-gcp-00-f6858b4e5c0c/energy_forecasting/dnn_normalized_fc/variables/


In [9]:
%%bash
TIMESTAMP=$(date -u +%Y%m%d_%H%M%S)
MODEL_DISPLAYNAME=energy_forecasting_dnn_normalized_fc_model
ENDPOINT_DISPLAYNAME=energy_forecasting_dnn_normalized_fc_model
IMAGE_URI="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest"
MODEL_LOCATION="gs://${PROJECT}/energy_forecasting/dnn_normalized_fc/"
echo "MODEL_LOCATION=${MODEL_LOCATION}"

# Model
MODEL_RESOURCENAME=$(gcloud ai models upload \
    --region=$REGION \
    --display-name=$MODEL_DISPLAYNAME \
    --container-image-uri=$IMAGE_URI \
    --artifact-uri=$MODEL_LOCATION \
    --format="value(model)")

MODEL_ID=$(echo $MODEL_RESOURCENAME | cut -d"/" -f6)

echo "MODEL_DISPLAYNAME=${MODEL_DISPLAYNAME}"
echo "MODEL_RESOURCENAME=${MODEL_RESOURCENAME}"
echo "MODEL_ID=${MODEL_ID}"

# Endpoint
ENDPOINT_RESOURCENAME=$(gcloud ai endpoints create \
  --region=$REGION \
  --display-name=$ENDPOINT_DISPLAYNAME \
  --format="value(name)")

ENDPOINT_ID=$(echo $ENDPOINT_RESOURCENAME | cut -d"/" -f6)

echo "ENDPOINT_DISPLAYNAME=${ENDPOINT_DISPLAYNAME}"
echo "ENDPOINT_RESOURCENAME=${ENDPOINT_RESOURCENAME}"
echo "ENDPOINT_ID=${ENDPOINT_ID}"

# Deployment
DEPLOYEDMODEL_DISPLAYNAME=${MODEL_DISPLAYNAME}_deployment
MACHINE_TYPE=n1-standard-2
MIN_REPLICA_COUNT=1
MAX_REPLICA_COUNT=3

gcloud ai endpoints deploy-model $ENDPOINT_RESOURCENAME \
  --region=$REGION \
  --model=$MODEL_RESOURCENAME \
  --display-name=$DEPLOYEDMODEL_DISPLAYNAME \
  --machine-type=$MACHINE_TYPE \
  --min-replica-count=$MIN_REPLICA_COUNT \
  --max-replica-count=$MAX_REPLICA_COUNT \
  --traffic-split=0=100

MODEL_LOCATION=gs://qwiklabs-gcp-00-f6858b4e5c0c/energy_forecasting/dnn_normalized_fc/
MODEL_DISPLAYNAME=energy_forecasting_dnn_normalized_fc_model
MODEL_RESOURCENAME=projects/186845041058/locations/us-east1/models/3586307862642556928
MODEL_ID=3586307862642556928
ENDPOINT_DISPLAYNAME=energy_forecasting_dnn_normalized_fc_model
ENDPOINT_RESOURCENAME=projects/186845041058/locations/us-east1/endpoints/2620320926943674368
ENDPOINT_ID=2620320926943674368


Using endpoint [https://us-east1-aiplatform.googleapis.com/]
Waiting for operation [6734220648081522688]...
....................done.
Using endpoint [https://us-east1-aiplatform.googleapis.com/]
Waiting for operation [3336254729230483456]...
...................done.
Created Vertex AI endpoint: projects/186845041058/locations/us-east1/endpoints/2620320926943674368.
Using endpoint [https://us-east1-aiplatform.googleapis.com/]
Waiting for operation [7794818360327274496]...
.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................